<a href="https://colab.research.google.com/github/perrijuan/UFRJ-Analytica-processo-seletivo/blob/main/abstract_ou_rascunho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importar bibliotecas de regressão (linear e logística), de score que de regressões, de estatística

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score,ConfusionMatrixDisplay

Com a biblioteca pandas, vou trazer o arquivo pro python

In [ ]:
campeonatos = pd.read_csv("campeonatos_futebol_atualizacao.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'campeonatos_futebol_atualizacao.csv'

Começando a análise exploratória dos dados, vou printar a head ( 5 primeiras linhas do dataset ), depois printar o .dtypes ( para analisar o tipo de dado que vamos tratar ) e vou usar o .info para ver quantos dados faltam

In [ ]:
print(campeonatos.head())

In [ ]:
print(campeonatos.dtypes)

In [ ]:
print(campeonatos.info())

Agora, vou ver onde tem valores faltando ( ou seja, NAN ), pra isso vou usar o isnull do pandas

In [ ]:
colunas_com_null = campeonatos.columns[campeonatos.isnull().any()]
print(colunas_com_null)

Só para conferir se todas colunas tem valor null:

In [ ]:
print(len(colunas_com_null) == len(campeonatos))

NameError: name 'colunas_com_null' is not defined

Agora eu vou tratar as colunas numéricas que tem valor NULL, vou tentar substituir com a mediana, mas fiquem a vontade para mudar. Para isso, vou criar uma df que tenham só as colunas numéricas que tem valor null.

In [ ]:
colunas_numericas_null = campeonatos[colunas_com_null].select_dtypes(include='number').columns


Ao inves de fazer um por um, vou fazer um for loop que vá substituindo os valores faltantes de cada coluna pela mediana.

In [ ]:
for coluna in colunas_numericas_null:

    campeonatos[coluna] = campeonatos[coluna].fillna(np.median(campeonatos[coluna]))

Para as colunas que são do tipo 'object', eu vou fazer a mesma coisa, eu vou tentar substituir com a moda, para evitar perdermos linhas do dataframe

In [ ]:
colunas_object_null = campeonatos[colunas_com_null].select_dtypes(include = 'object').columns

for coluna in colunas_object_null:

  moda = campeonatos[coluna].mode()[0]
  campeonatos[coluna] = campeonatos[coluna].fillna(moda)

Vou identificar outliers utilizando zscore, ou seja, para modulo de zscore > 3, existe um outlier.

In [ ]:

colunas_numericas = campeonatos.select_dtypes(include = 'number')
z_scores = stats.zscore(colunas_numericas)
outliers = (np.abs(z_scores) > 3)

for coluna in colunas_numericas.columns:
    if outliers[coluna].any():
        print(f"Outliers na coluna '{coluna}':")


Outliers na coluna 'Cartões amarelos 1':
Outliers na coluna 'Cartões amarelos 2':
Outliers na coluna 'Cartões vermelhos 1':
Outliers na coluna 'Cartões vermelhos 2':
Outliers na coluna 'Gols 1':
Outliers na coluna 'Gols 2':


Função que limpa os dados de um dataframe.

In [ ]:

def dataset_limpo_para_csv(df, nome_arquivo='dataset_limpo.csv'):
    colunas_com_null = df.columns[df.isnull().any()]

    colunas_numericas_null = df[colunas_com_null].select_dtypes(include='number').columns
    for coluna in colunas_numericas_null:
        df[coluna] = df[coluna].fillna(np.median(df[coluna]))

    colunas_object_null = df[colunas_com_null].select_dtypes(include='object').columns
    for coluna in colunas_object_null:
        moda = df[coluna].mode()[0]
        df[coluna] = df[coluna].fillna(moda)

    colunas_numericas = df.select_dtypes(include='number')
    z_scores = stats.zscore(colunas_numericas)
    outliers = (np.abs(z_scores) > 3)
    linhas_sem_outliers = ~(outliers.any(axis=1))
    df = df[linhas_sem_outliers]

    df.to_csv(nome_arquivo, index=False)

    return df


Vou criar duas funções para padronizar os dados com o StandarScaler e outra


 ------------------------------------------------

In [ ]:

def padronizar_dados(df):
    scaler = StandardScaler()
    col_num = df.select_dtypes(include='number').columns
    df_standard = df.copy()
    df_standard[col_num] = scaler.fit_transform(df[col_num])

    return df_standard


# algoritimos de ml e implementação de metricas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
import io
from abc import ABC, abstractmethod
from typing import Dict, List, Tuple, Optional, Union
import pickle

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                            f1_score, roc_curve, auc, confusion_matrix,
                            classification_report, roc_auc_score)
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Verificar disponibilidade do XGBoost -> a consertar
try:
    from xgboost import XGBClassifier
    xgboost_available = True
except ImportError:
    xgboost_available = False
    print("XGBoost não está instalado. Para instalar, execute: !pip install xgboost")

# Configurações globais de visualização
plt.style.use('fivethirtyeight')
sns.set(style='whitegrid', palette='muted', font_scale=1.2)


class DataLoader:
    """Classe responsável pelo carregamento de dados."""

    @staticmethod
    def load_from_upload() -> pd.DataFrame:
        """Carrega dados via upload no Google Colab."""
        print("Por favor, faça upload do arquivo CSV:")
        uploaded = files.upload()

        for fn in uploaded.keys():
            print(f'Arquivo carregado: {fn}')
            df = pd.read_csv(io.BytesIO(uploaded[fn]))
            return df

        raise ValueError("Nenhum arquivo foi carregado.")

    @staticmethod
    def load_from_drive(path: str) -> pd.DataFrame:
        """Carrega dados do Google Drive."""
        from google.colab import drive
        drive.mount('/content/drive')
        df = pd.read_csv(path)
        return df


class DataProcessor:
    """Classe responsável pelo pré-processamento de dados."""

    def __init__(self, random_state: int = 42):
        self.random_state = random_state
        self.scaler = StandardScaler()
        self.feature_names = None

    def preprocess(self, df: pd.DataFrame, target_column: str, test_size: float = 0.25) -> Tuple:
        """Pré-processa os dados para treino e teste."""
        # Verificar e tratar valores nulos
        self._handle_missing_values(df)

        # Separar recursos e alvo
        X = df.drop(target_column, axis=1)
        y = df[target_column]

        # Codificar variáveis categóricas
        X = self._encode_categorical_features(X)
        self.feature_names = X.columns

        # Dividir em conjuntos de treino e teste
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=self.random_state, stratify=y
        )

        # Escalar características
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)

        return X_train_scaled, X_test_scaled, y_train, y_test

    def _handle_missing_values(self, df: pd.DataFrame) -> None:
        """Trata valores ausentes no DataFrame."""
        if df.isnull().sum().sum() > 0:
            print("Dados contêm valores nulos. Realizando tratamento...")

            # Preencher valores numéricos nulos com a média
            num_cols = df.select_dtypes(include=[np.number]).columns
            for col in num_cols:
                if df[col].isnull().sum() > 0:
                    df[col].fillna(df[col].mean(), inplace=True)

            # Preencher valores categóricos nulos com a moda
            cat_cols = df.select_dtypes(exclude=[np.number]).columns
            for col in cat_cols:
                if df[col].isnull().sum() > 0:
                    df[col].fillna(df[col].mode()[0], inplace=True)

    def _encode_categorical_features(self, X: pd.DataFrame) -> pd.DataFrame:
        """Codifica variáveis categóricas usando one-hot encoding."""
        return pd.get_dummies(X)


class Model(ABC):
    """Classe base abstrata para modelos de classificação."""

    def __init__(self, name: str, model: BaseEstimator):
        self.name = name
        self.model = model
        self.metrics = {}
        self.roc_data = None

    def fit(self, X_train, y_train):
        """Treina o modelo com os dados fornecidos."""
        self.model.fit(X_train, y_train)
        return self

    def predict(self, X):
        """Faz previsões com o modelo treinado."""
        return self.model.predict(X)

    def predict_proba(self, X):
        """Retorna as probabilidades para cada classe."""
        return self.model.predict_proba(X)

    def get_name(self) -> str:
        """Retorna o nome do modelo."""
        return self.name

    def get_metrics(self) -> Dict:
        """Retorna as métricas calculadas do modelo."""
        return self.metrics

    def get_roc_data(self) -> pd.DataFrame:
        """Retorna os dados da curva ROC."""
        return self.roc_data

#Fabrica para instanciar n modelos gerando nossa comparação muito mais solida e robusta

class ModelFactory:
    """Fábrica para criar instâncias de modelos."""

    @staticmethod
    def create_models() -> List[Model]:
        """Cria e retorna uma lista de todos os modelos disponíveis."""
        models = []

        # Criar instâncias de cada modelo
        models.append(Model("Regressão Logística",
                           LogisticRegression(max_iter=1000, random_state=42)))

        models.append(Model("Árvore de Decisão",
                           DecisionTreeClassifier(max_depth=5, random_state=42)))

        models.append(Model("KNN",
                           KNeighborsClassifier(n_neighbors=5)))

        models.append(Model("Naive Bayes (Gaussian)",
                           GaussianNB()))

        models.append(Model("SVM",
                           SVC(probability=True, random_state=42)))

        models.append(Model("Rede Neural",
                           MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)))

        models.append(Model("Random Forest",
                           RandomForestClassifier(n_estimators=100, random_state=42)))

        models.append(Model("Gradient Boosting",
                           GradientBoostingClassifier(n_estimators=100, random_state=42)))

        # Adicionar XGBoost se disponível
        if xgboost_available:
            models.append(Model("XGBoost",
                               XGBClassifier(n_estimators=100, random_state=42)))

        return models


class ModelEvaluator:
    """Classe responsável pela avaliação de modelos."""

    @staticmethod
    def evaluate_model(model: Model, X_test, y_test) -> Model:
        """Avalia um modelo e calcula suas métricas de desempenho."""
        # Fazer previsões
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]

        # Calcular métricas básicas
        metrics = {
            "Model": model.get_name(),
            "Accuracy": accuracy_score(y_test, y_pred),
            "Precision": precision_score(y_test, y_pred, average='weighted'),
            "Recall": recall_score(y_test, y_pred, average='weighted'),
            "F1 Score": f1_score(y_test, y_pred, average='weighted')
        }

        # Calcular curva ROC e AUC
        fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
        metrics["ROC AUC"] = auc(fpr, tpr)

        # Armazenar dados da curva ROC
        model.roc_data = pd.DataFrame({
            'FPR': fpr,
            'TPR': tpr,
            'Model': [model.get_name()] * len(fpr)
        })

        # Salvar métricas no modelo
        model.metrics = metrics

        # Imprimir relatório de classificação
        print(f"\n--- Relatório de Classificação: {model.get_name()} ---")
        print(classification_report(y_test, y_pred))
        print("Matriz de Confusão:")
        cm = confusion_matrix(y_test, y_pred)
        print(cm)
        print("-" * 50)

        return model


class ResultVisualizer:
    """Classe responsável por visualizar os resultados da avaliação de modelos."""

    def __init__(self):
        self.colors = sns.color_palette('viridis', 10)

    def create_comparison_plots(self, results_df: pd.DataFrame, roc_results: pd.DataFrame):
        """Cria todos os gráficos de comparação entre modelos."""
        self._plot_metrics_comparison(results_df)
        self._plot_roc_auc_comparison(results_df)
        self._plot_roc_curves(results_df, roc_results)
        self._plot_metrics_heatmap(results_df)
        self._plot_radar_chart(results_df)

    def _plot_metrics_comparison(self, results_df: pd.DataFrame):
        """Plota comparação de métricas (Acurácia, Precisão, Revocação, F1)."""
        plt.figure(figsize=(14, 8))
        metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']

        results_melted = pd.melt(results_df,
                                id_vars=['Model'],
                                value_vars=metrics,
                                var_name='Metric',
                                value_name='Score')

        sns.barplot(x='Model', y='Score', hue='Metric', data=results_melted)
        plt.title('Comparação de Métricas entre Modelos', fontsize=16)
        plt.xlabel('Modelo', fontsize=14)
        plt.ylabel('Pontuação', fontsize=14)
        plt.xticks(rotation=45, ha='right')
        plt.legend(title='Métrica')
        plt.tight_layout()
        plt.show()

    def _plot_roc_auc_comparison(self, results_df: pd.DataFrame):
        """Plota comparação de ROC AUC como gráfico de barras."""
        plt.figure(figsize=(12, 6))
        ax = sns.barplot(x='Model', y='ROC AUC', data=results_df, palette=self.colors)
        plt.title('Comparação de ROC AUC entre Modelos', fontsize=16)
        plt.xlabel('Modelo', fontsize=14)
        plt.ylabel('ROC AUC', fontsize=14)
        plt.xticks(rotation=45, ha='right')

        # Adicionar valores nas barras
        for i, v in enumerate(results_df['ROC AUC']):
            ax.text(i, v+0.01, f'{v:.3f}', ha='center', fontsize=10)

        plt.tight_layout()
        plt.show()

    def _plot_roc_curves(self, results_df: pd.DataFrame, roc_results: pd.DataFrame):
        """Plota curvas ROC para todos os modelos."""
        plt.figure(figsize=(12, 8))

        # Plotar linha de referência (random classifier)
        plt.plot([0, 1], [0, 1], 'r--', label='Classificador Aleatório')

        # Plotar curva ROC para cada modelo
        for name in results_df['Model'].unique():
            model_data = roc_results[roc_results['Model'] == name]
            plt.plot(model_data['FPR'], model_data['TPR'],
                    label=f"{name} (AUC = {results_df.loc[results_df['Model'] == name, 'ROC AUC'].values[0]:.3f})")

        plt.title('Curvas ROC para Todos os Modelos', fontsize=16)
        plt.xlabel('Taxa de Falsos Positivos (1 - Especificidade)', fontsize=14)
        plt.ylabel('Taxa de Verdadeiros Positivos (Sensibilidade)', fontsize=14)
        plt.legend(loc='lower right')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    def _plot_metrics_heatmap(self, results_df: pd.DataFrame):
        """Plota heatmap das métricas para melhor visualização."""
        plt.figure(figsize=(12, 8))
        heatmap_data = results_df.set_index('Model')
        heatmap_data = heatmap_data[['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC']]

        sns.heatmap(heatmap_data, annot=True, cmap='viridis', fmt='.3f', linewidths=0.5)
        plt.title('Heatmap de Métricas por Modelo', fontsize=16)
        plt.tight_layout()
        plt.show()

    def _plot_radar_chart(self, results_df: pd.DataFrame):
        """Plota gráfico radar para métricas por modelo."""
        # Preparar dados para gráfico radar
        categories = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC']
        model_names = results_df['Model'].tolist()

        # Configuração do gráfico
        fig = plt.figure(figsize=(12, 10))

        # Número de categorias
        N = len(categories)

        # Ângulo para cada eixo
        angles = [n / float(N) * 2 * np.pi for n in range(N)]
        angles += angles[:1]  # Fechar o círculo

        # Plotar para cada modelo
        ax = plt.subplot(111, polar=True)

        # Adicionar linhas do grid circular
        plt.xticks(angles[:-1], categories, size=12)
        ax.set_rlabel_position(0)
        plt.yticks([0.2, 0.4, 0.6, 0.8, 1.0], ["0.2", "0.4", "0.6", "0.8", "1.0"],
                  color="grey", size=10)
        plt.ylim(0, 1)

        # Plotar cada modelo
        for i, model in enumerate(model_names):
            values = results_df.loc[results_df['Model'] == model, categories].values.flatten().tolist()
            values += values[:1]  # Fechar o círculo
            ax.plot(angles, values, linewidth=2, linestyle='solid', label=model)
            ax.fill(angles, values, alpha=0.1)

        plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
        plt.title('Gráfico Radar de Métricas por Modelo', size=18)
        plt.tight_layout()
        plt.show()


class ModelComparisonSystem:
    """Classe principal que orquestra todo o processo de comparação de modelos."""

    def __init__(self, random_state: int = 42):
        self.data_loader = DataLoader()
        self.data_processor = DataProcessor(random_state=random_state)
        self.model_factory = ModelFactory()
        self.model_evaluator = ModelEvaluator()
        self.result_visualizer = ResultVisualizer()

        # Armazenar resultados
        self.df = None
        self.target_column = None
        self.feature_names = None
        self.models = []
        self.results_df = None
        self.roc_results = None

    def load_data(self, option: int = 1, drive_path: str = None) -> pd.DataFrame:
        """Carrega dados conforme a opção escolhida."""
        if option == 1:
            self.df = self.data_loader.load_from_upload()
        elif option == 2 and drive_path:
            self.df = self.data_loader.load_from_drive(drive_path)
        else:
            raise ValueError("Opção inválida ou caminho do drive não especificado.")

        return self.df

    def analyze_data(self) -> None:
        """Analisa os dados e mostra estatísticas básicas."""
        if self.df is None:
            raise ValueError("Os dados não foram carregados. Use load_data() primeiro.")

        print("\nPrimeiras linhas do dataset:")
        print(self.df.head())

        print("\nInformações do dataset:")
        print(self.df.info())

        print("\nEstatísticas descritivas:")
        print(self.df.describe())

    def select_target(self, target_index: int) -> str:
        """Seleciona a coluna alvo para predição."""
        if self.df is None:
            raise ValueError("Os dados não foram carregados. Use load_data() primeiro.")

        self.target_column = self.df.columns[target_index]
        print(f"\nColuna alvo selecionada: {self.target_column}")
        return self.target_column

    def preprocess_data(self, test_size: float = 0.25) -> Tuple:
        """Pré-processa os dados para modelagem."""
        if self.df is None or self.target_column is None:
            raise ValueError("Dados ou coluna alvo não definidos.")

        X_train, X_test, y_train, y_test = self.data_processor.preprocess(
            self.df, self.target_column, test_size
        )
        self.feature_names = self.data_processor.feature_names

        return X_train, X_test, y_train, y_test

    def train_and_evaluate_models(self, X_train, X_test, y_train, y_test) -> pd.DataFrame:
        """Treina e avalia todos os modelos."""
        # Obter modelos
        self.models = self.model_factory.create_models()

        # Treinar e avaliar cada modelo
        for model in self.models:
            print(f"Treinando {model.get_name()}...")
            model.fit(X_train, y_train)
            self.model_evaluator.evaluate_model(model, X_test, y_test)

        # Coletar resultados
        results_list = [model.get_metrics() for model in self.models]
        self.results_df = pd.DataFrame(results_list)

        # Coletar dados da curva ROC
        roc_results_list = [model.get_roc_data() for model in self.models]
        self.roc_results = pd.concat(roc_results_list, ignore_index=True)

        return self.results_df

    def visualize_results(self) -> None:
        """Visualiza os resultados da comparação de modelos."""
        if self.results_df is None or self.roc_results is None:
            raise ValueError("Resultados não disponíveis. Execute train_and_evaluate_models() primeiro.")

        self.result_visualizer.create_comparison_plots(self.results_df, self.roc_results)

    def save_results(self, filename: str = 'modelo_comparacao_resultados.csv') -> None:
        """Salva os resultados em um arquivo CSV."""
        if self.results_df is None:
            raise ValueError("Resultados não disponíveis. Execute train_and_evaluate_models() primeiro.")

        self.results_df.to_csv(filename, index=False)
        print(f"\nResultados salvos em '{filename}'")

    def save_models(self, directory: str = 'models') -> None:
        """Salva os modelos treinados."""
        import os

        if not os.path.exists(directory):
            os.makedirs(directory)

        for model in self.models:
            with open(f"{directory}/{model.get_name().replace(' ', '_').lower()}.pkl", 'wb') as f:
                pickle.dump(model.model, f)

        print(f"\nModelos salvos no diretório '{directory}'")

    def get_best_model(self, metric: str = 'F1 Score') -> Model:
        """Retorna o melhor modelo baseado na métrica especificada."""
        if self.results_df is None:
            raise ValueError("Resultados não disponíveis. Execute train_and_evaluate_models() primeiro.")

        best_model_name = self.results_df.loc[self.results_df[metric].idxmax(), 'Model']

        for model in self.models:
            if model.get_name() == best_model_name:
                return model

        raise ValueError(f"Modelo '{best_model_name}' não encontrado.")


def run_interactive():
    """Executa o sistema de comparação de modelos de forma interativa."""
    system = ModelComparisonSystem()

    print("Bem-vindo à análise comparativa de modelos de Machine Learning!")
    print("\n1. Carregar dados via upload")
    print("2. Carregar dados do Google Drive")

    choice = int(input("\nEscolha uma opção (1/2): "))

    if choice == 1:
        system.load_data(option=1)
    elif choice == 2:
        path = input("Insira o caminho para o arquivo CSV no Google Drive: ")
        system.load_data(option=2, drive_path=path)
    else:
        print("Opção inválida. Usando upload padrão.")
        system.load_data(option=1)

    # Analisar dados
    system.analyze_data()

    # Selecionar coluna alvo
    print("\nColunas disponíveis:")
    for i, col in enumerate(system.df.columns):
        print(f"{i}: {col}")

    target_idx = int(input("\nEscolha o índice da coluna alvo (variável dependente): "))
    system.select_target(target_idx)

    # Pré-processar dados
    X_train, X_test, y_train, y_test = system.preprocess_data()

    # Treinar e avaliar modelos
    system.train_and_evaluate_models(X_train, X_test, y_train, y_test)

    # Visualizar resultados
    system.visualize_results()

    # Mostrar resultados finais
    print("\nResultados finais:")
    sorted_results = system.results_df.sort_values('F1 Score', ascending=False)
    print(sorted_results)

    # Salvar resultados
    save_option = input("\nDeseja salvar os resultados? (s/n): ")
    if save_option.lower() == 's':
        system.save_results()

    # Salvar modelos
    save_models_option = input("\nDeseja salvar os modelos treinados? (s/n): ")
    if save_models_option.lower() == 's':
        system.save_models()

    # Mostrar melhor modelo
    best_metric = input("\nQual métrica você deseja usar para selecionar o melhor modelo? (Accuracy/Precision/Recall/F1 Score/ROC AUC): ")
    try:
        best_model = system.get_best_model(best_metric)
        print(f"\nMelhor modelo baseado em {best_metric}: {best_model.get_name()}")
        print(f"Métricas do melhor modelo: {best_model.get_metrics()}")
    except:
        print("Métrica inválida. Usando F1 Score como padrão.")
        best_model = system.get_best_model()
        print(f"\nMelhor modelo baseado em F1 Score: {best_model.get_name()}")
        print(f"Métricas do melhor modelo: {best_model.get_metrics()}")


if __name__ == "__main__":
    run_interactive()

Bem-vindo à análise comparativa de modelos de Machine Learning!

1. Carregar dados via upload
2. Carregar dados do Google Drive
